In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyodesys.symbolic import SymbolicSys
%matplotlib inline

In [ ]:
def dydt(t, y, params, backend):
    return [y[1], -y[0]*params[0]]  # y'' = -k y => y = C1 + sin(k**0.5 * x + C2)

In [ ]:
odesys = SymbolicSys.from_callback(dydt, 2, 1)
odesys.nonlinear_invariants = [odesys.params[0]*odesys.dep[0]**2/2 + odesys.dep[1]**2/2]
A0, spring_const = 3, 5
res = odesys.integrate(17, [A0, 0], [spring_const], integrator='cvode', method='adams', nsteps=2000)
print({k: v for k, v in res.info.items() if not k.startswith('internal_')})

In [ ]:
def analytic(x):
    return A0*np.cos(spring_const**.5*x)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))
res.plot(ax=axes[0])

lo, hi = 1.27, 1.52
xplt = np.linspace(lo, hi)
axes[1].plot(xplt, analytic(xplt), ls=':')
res.plot(ax=axes[1], between=(lo, hi), indices=(0,))
axes[1].axvline(spring_const**-.5 * np.pi, 0, 1) #, transform=axes[1].get_xaxis_transform())
axes[1].axvline(res.xout[np.argmin(res.yout[res.xout<2, 0])], 0, 1, linestyle='--') # transform=axes[1].get_xaxis_transform(), 
x_select, y_select = res.between(lo, hi)
axes[2].plot(x_select, y_select[:, 0] - analytic(x_select))
plt.tight_layout()

In [ ]:
x_at = np.linspace(lo, hi)
plt.plot(x_at, [res.at(_)[0] for _ in x_at])

In [ ]:
import scipy; print(scipy.__version__)

In [ ]:
_ = res.plot_invariant_violations()

In [ ]:
import matplotlib.pyplot as plt
odesys.nonlinear_invariant_names = ['energy']
_ = res.plot_invariant_violations()
_ = plt.legend(loc='best')